# ResNet 18

In [1]:
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
from torch.optim import lr_scheduler
import copy
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch.optim as optim
import random
import torch.nn as nn
import torch.nn.functional as F
import pprint, pickle
from torchvision import datasets, models, transforms
import time
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image


from __future__ import print_function, division

from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [2]:
"""
i:166 (* 8)
o:82  (*16)
p:1330
u:83  (*16)
""" 
def data_organizer():    
    #store training img address
    txt = './8536_data/newDeepSmoothing/training_data/list1.txt' #data_list
    fh = open(txt, 'r')
    imgs = []
    i = 0
    cnt = 0
    points1 = np.zeros([1661,]).tolist()
    for line in fh:
        line = line.strip('\n')
        line = line.rstrip()
        words = line.split()
        if str(words[1]) == "i":
            label = 0
            cnt = 8
        elif str(words[1]) == "o":
            label = 1
            cnt = 16
        elif str(words[1]) == "u":
            label = 2
            cnt = 16
        else:
            label = 3
            cnt = 1
            
        for j in range(cnt):
#             imgs.append((words[0],str(words[1])))
            imgs.append((words[0],str(label)))
            points1[i] = words[0]
            i = i+1
        
    file = open('data.pickle', 'wb') #data_img
    pickle.dump(points1, file)
    file.close()

    #store validation img address
    txt = 'D:\\Test_Data2\\list.txt' #
    fh = open(txt, 'r')
    imgs = []
    i = 0
    points1 = np.zeros([1661,]).tolist()
    for line in fh:
        line = line.strip('\n')
        line = line.rstrip()
        words = line.split()

        imgs.append((words[0],str(words[1])))
        points1[i] = words[0]
        i = i+1
    file = open('valimg.pickle', 'wb')
    pickle.dump(points1, file)
    file.close()
    

In [3]:
"""
i:166 (* 8)
o:82  (*16)
p:1330
u:83  (*16)
""" 

def default_loader(path):
    pkl_file = open(path, 'rb')
    data1 = pickle.load(pkl_file)
    data1 = data1/256
    pkl_file.close()
    return data1
class MyDataset(Dataset):
    def __init__(self, txt, transform=None, target_transform=None, loader=default_loader):
        fh = open(txt, 'r')
        imgs = []
        for line in fh:
            line = line.strip('\n')
            line = line.rstrip()
            words = line.split()
            if str(words[1]) == "i":
                label = 0
                cnt = 8
            elif str(words[1]) == "o":
                label = 1
                cnt = 16
            elif str(words[1]) == "u":
                label = 2
                cnt = 16
            else:
                label = 3
                cnt = 1

            for j in range(cnt):
    #             imgs.append((words[0],str(words[1])))
                imgs.append((words[0],str(label)))
#                 points1[i] = words[0]
#                 i = i+1
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = self.loader(fn)
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

In [17]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize(mean = [0.5]*30, std = [0.5]*30)
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

trainset = MyDataset(txt='D:\\Test_Data2\\list.txt',transform=data_transforms['train'])

train_size = 0.70
val_size = 0.20
test_size = 0.10
num_train = len(trainset)
indices = list(range(num_train))
random.shuffle(indices)
split = int(np.floor(train_size * num_train))
split_val = int(np.floor(val_size * num_train))

train_idx, valid_idx, test_idx = indices[:split], indices[split:split_val+split], indices[split_val+split:]

image_datasets={'train': MyDataset(txt='D:\\Test_Data2\\list.txt',  transform=data_transforms['train']),
                'val': MyDataset(txt='D:\\Test_Data2\\list.txt',  transform=data_transforms['val']),
                'test': MyDataset(txt='D:\\Test_Data2\\list.txt',  transform=data_transforms['test']),
                }
train_sampler = torch.utils.data.sampler.RandomSampler(train_idx)
val_sampler = torch.utils.data.sampler.RandomSampler(valid_idx)
test_sampler = torch.utils.data.sampler.RandomSampler(test_idx)

dataloaders = {
        'train': torch.utils.data.DataLoader(trainset, batch_size=4,sampler = train_sampler,
                                          shuffle=False, num_workers=0),
        'val': torch.utils.data.DataLoader(trainset, batch_size=4,sampler = val_sampler,
                                          shuffle=False, num_workers=0),
        'test': torch.utils.data.DataLoader(trainset, batch_size=1,sampler = test_sampler,
                                          shuffle=False, num_workers=0),
                }
dataset_sizes = {
        'train':num_train* train_size,
        'val': num_train* val_size,
        'test': num_train * test_size
				    }

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    print(np.max(inp))
    mean = np.array([0.5])
    std = np.array([0.5])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    print(np.max(inp))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # 暂停一会, 让 plots 更新

# 获得一批训练数据
# inputs, classes = next(iter(dataloaders['train']))
inputs, classes = next(iter(dataloaders['train']))
print(np.shape(inputs))
print(classes)
# print(np.max(inputs[0,:,:,:]))
# 从这批数据生成一个方格
# out = torchvision.utils.make_grid(inputs[:,0,:,:], title=[classes[x] for x in classes])

# imshow(out)

torch.Size([4, 30, 1024, 1024])
('1', '3', '1', '1')


In [20]:
print(inputs)

tensor([[[[ 0.0021,  0.0021,  0.0021,  ...,  0.0016,  0.0017,  0.0014],
          [ 0.0023,  0.0020,  0.0020,  ...,  0.0018,  0.0016,  0.0016],
          [ 0.0020,  0.0019,  0.0019,  ...,  0.0016,  0.0017,  0.0015],
          ...,
          [ 0.0018,  0.0018,  0.0021,  ...,  0.0017,  0.0015,  0.0017],
          [ 0.0019,  0.0019,  0.0019,  ...,  0.0018,  0.0016,  0.0017],
          [ 0.0017,  0.0020,  0.0020,  ...,  0.0014,  0.0016,  0.0016]],

         [[ 0.0021,  0.0020,  0.0020,  ...,  0.0016,  0.0017,  0.0015],
          [ 0.0021,  0.0020,  0.0019,  ...,  0.0014,  0.0015,  0.0016],
          [ 0.0019,  0.0019,  0.0019,  ...,  0.0015,  0.0016,  0.0016],
          ...,
          [ 0.0019,  0.0020,  0.0019,  ...,  0.0016,  0.0016,  0.0017],
          [ 0.0019,  0.0019,  0.0017,  ...,  0.0017,  0.0018,  0.0016],
          [ 0.0019,  0.0020,  0.0018,  ...,  0.0016,  0.0014,  0.0017]],

         [[ 0.0021,  0.0021,  0.0019,  ...,  0.0017,  0.0029,  0.0019],
          [ 0.0021,  0.0022,  

In [27]:
a = default_loader('D://Test_Data2//o2005102500.pkl')
print(a)

[[[0.5        0.5390625  0.5234375  ... 0.5        0.5078125  0.453125  ]
  [0.5234375  0.50390625 0.51171875 ... 0.48046875 0.48828125 0.53125   ]
  [0.44140625 0.4609375  0.48828125 ... 0.47265625 0.50390625 0.48046875]
  ...
  [0.4453125  0.3828125  0.4609375  ... 0.4375     0.4609375  0.4296875 ]
  [0.7421875  0.68359375 0.7109375  ... 0.59765625 0.58984375 0.58203125]
  [0.4765625  0.48828125 0.45703125 ... 0.42578125 0.4375     0.4296875 ]]

 [[0.60546875 0.53515625 0.609375   ... 0.58203125 0.55078125 0.6015625 ]
  [0.47265625 0.4921875  0.5625     ... 0.49609375 0.4765625  0.52734375]
  [0.46484375 0.484375   0.49609375 ... 0.453125   0.46875    0.5390625 ]
  ...
  [0.4765625  0.48828125 0.5078125  ... 0.46875    0.44140625 0.453125  ]
  [0.48046875 0.5        0.51171875 ... 0.44140625 0.4609375  0.5       ]
  [0.5        0.48828125 0.5        ... 0.46484375 0.4765625  0.43359375]]

 [[0.5        0.546875   0.55859375 ... 0.5625     0.53515625 0.5       ]
  [0.5        0.511718

In [26]:
print(np.max(a))

237.62890625


In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [8]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [9]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [97]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


AttributeError: 'tuple' object has no attribute 'to'